<a href="https://colab.research.google.com/github/joaogabrielcolares/algoritimo-genetivo/blob/main/genetic-algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Descrição do Problema:**
Investidores e gestores de fundos enfrentam o desafio de criar portfólios de investimento que maximizem o retorno esperado enquanto minimizam o risco. A composição do portfólio deve equilibrar diferentes ativos financeiros, como ações, títulos, commodities e outros instrumentos financeiros, de acordo com as preferências e restrições do investidor.

**Objetivos:**
Maximizar o Retorno Esperado: O objetivo é selecionar uma combinação de ativos que ofereça o maior retorno possível com base em previsões e análises de mercado.
Minimizar o Risco: Reduzir a volatilidade e o risco associado ao portfólio, diversificando os investimentos para proteger contra perdas significativas.
Equilibrar Preferências e Restrições: Levar em consideração as preferências do investidor, como tolerância ao risco, horizonte de investimento e restrições específicas (por exemplo, evitar certos setores ou tipos de ativos).
Critérios de Sucesso:
Aumento do Retorno sobre o Investimento (ROI): O portfólio deve proporcionar um retorno financeiro superior ao de benchmarks ou índices de mercado relevantes.
Redução do Risco: O portfólio deve apresentar menor volatilidade e risco em comparação com alternativas menos diversificadas.
Satisfação do Investidor: O portfólio deve atender às expectativas e preferências do investidor, proporcionando uma experiência de investimento satisfatória.
Como Algoritmos Genéticos Podem Ajudar:
Algoritmos Genéticos (AGs) são uma técnica de otimização inspirada na evolução natural. Eles podem ser utilizados para resolver o problema de design de portfólios de investimento da seguinte maneira:

**Representação do Portfólio:** Cada indivíduo na população do AG representa um portfólio de investimento, codificado como um vetor de pesos que indicam a proporção de cada ativo no portfólio.

**Função de Fitness:** A função de fitness avalia a qualidade de cada portfólio com base em critérios como retorno esperado e risco. Uma função comum é a razão de Sharpe, que mede o retorno ajustado pelo risco.

**Operadores Genéticos:** Os operadores de seleção, cruzamento (crossover) e mutação são aplicados para gerar novas soluções (portfólios) a partir das existentes. A seleção favorece portfólios com melhor desempenho, enquanto o cruzamento e a mutação introduzem diversidade e exploram novas combinações de ativos.

**Evolução:** O AG itera através de várias gerações, refinando a população de portfólios até encontrar uma solução otimizada que atenda aos objetivos do investidor.

In [ ]:
import numpy as np

# todo
# Acho que seria legal pegar uma lista de ativos para usarmos na nossa comparação, ou podemos colocar randomico como está ai. https://www.kaggle.com/
# returns = np.random.randn(1000, 10)  # 1000 dias de retornos para 10 ativos
# best_portfolio = genetic_algorithm(returns)
# print("Melhor Portfólio:", best_portfolio)

In [1]:
# Função de fitness (razão de Sharpe)
def fitness(portfolio, returns, risk_free_rate=0.01):
    portfolio_return = np.dot(portfolio, returns.mean())
    portfolio_volatility = np.sqrt(np.dot(portfolio.T, np.dot(returns.cov(), portfolio)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return sharpe_ratio

In [ ]:
# Inicialização da população
def initialize_population(pop_size, num_assets):
    return np.random.dirichlet(np.ones(num_assets), size=pop_size)

In [ ]:
# Seleção
def selection(population, fitness_scores):
    selected_indices = np.argsort(fitness_scores)[-len(population)//2:]
    return population[selected_indices]

In [ ]:
# Cruzamento
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1)-1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

In [ ]:
# Mutação
def mutation(portfolio, mutation_rate=0.01):
    if np.random.rand() < mutation_rate:
        mutation_point = np.random.randint(len(portfolio))
        portfolio[mutation_point] = np.random.rand()
        portfolio /= portfolio.sum()  # Normalizar para manter a soma dos pesos igual a 1
    return portfolio

In [ ]:
# Algoritmo Genético
def genetic_algorithm(returns, pop_size=100, generations=1000, mutation_rate=0.01):
    num_assets = returns.shape[1]
    population = initialize_population(pop_size, num_assets)

    for generation in range(generations):
        fitness_scores = np.array([fitness(portfolio, returns) for portfolio in population])
        population = selection(population, fitness_scores)

        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = population[np.random.choice(len(population), 2, replace=False)]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1, mutation_rate))
            new_population.append(mutation(child2, mutation_rate))

        population = np.array(new_population)

    best_portfolio = population[np.argmax([fitness(portfolio, returns) for portfolio in population])]
    return best_portfolio

In [ ]:
best_portfolio = genetic_algorithm()
print("Melhor Portfólio:", best_portfolio)